In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import spacy
import sys
import gpt3
import cross_coref
import entity_linkage

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dgg32\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
error loading _jsonnet (this is expected on Windows), treating C:\Users\dgg32\AppData\Local\Temp\tmp_dhkurff\config.json as plain json
Some weights of the model checkpoint at microsoft/infoxlm-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassifica

In [2]:
tokenizer = AutoTokenizer.from_pretrained("pucpr/clinicalnerpt-chemical")

model = AutoModelForTokenClassification.from_pretrained("pucpr/clinicalnerpt-chemical")

model_infer = pipeline('ner',model=model,tokenizer=tokenizer)

nlp = spacy.load("en_core_web_sm")

In [3]:
done = set()
is_header = False

for line in open("./raw_data/cazy_gpt3_extract.tsv", 'r'):
    if is_header == False:
        is_header = True
    else:
        done.add(line.split("\t")[0])

done

{'CBM0',
 'CBM1',
 'CBM10',
 'CBM11',
 'CBM12',
 'CBM13',
 'CBM14',
 'CBM15',
 'CBM16',
 'CBM17',
 'CBM18',
 'CBM19',
 'CBM2',
 'CBM20',
 'CBM21',
 'CBM22',
 'CBM23',
 'CBM24',
 'CBM25',
 'CBM26',
 'CBM27',
 'CBM28',
 'CBM29',
 'CBM3',
 'CBM30',
 'CBM32',
 'CBM33',
 'CBM34',
 'CBM35',
 'CBM36',
 'CBM37',
 'CBM38',
 'CBM39',
 'CBM4',
 'CBM40',
 'CBM41',
 'CBM42',
 'CBM43',
 'CBM44',
 'CBM45',
 'CBM46',
 'CBM47',
 'CBM48',
 'CBM49',
 'CBM5',
 'CBM50',
 'CBM51',
 'CBM52',
 'CBM53',
 'CBM54',
 'CBM55',
 'CBM56',
 'CBM57',
 'CBM58',
 'CBM59',
 'CBM6',
 'CBM60',
 'CBM61',
 'CBM62',
 'CBM63',
 'CBM64',
 'CBM65',
 'CBM66',
 'CBM67',
 'CBM68',
 'CBM69',
 'CBM7',
 'CBM70',
 'CBM71',
 'CBM72',
 'CBM73',
 'CBM74',
 'CBM75',
 'CBM76',
 'CBM77',
 'CBM78',
 'CBM79',
 'CBM8',
 'CBM80',
 'CBM81',
 'CBM82',
 'CBM83',
 'CBM84',
 'CBM87',
 'CBM88',
 'CBM9',
 'CBM90'}

In [5]:
input_file = "./raw_data/cazy_activity.tsv"


with open(input_file, 'r', newline="", encoding='utf-8') as infile:

    for line in infile:
        fields = line.split("\t")
        cazy = fields[0]
        raw_text = fields[1]

        #if cazy == "CBM80":
        if cazy.startswith("CBM") and len(raw_text.strip()) > 0 and cazy not in done:
            #print (cazy, raw_text)
            sentences = (cross_coref.resolve_pronoun(raw_text))

            #print (sentences)

            #print (f'"{sent.text.strip()}"')
            res = gpt3.extract_relation("gpt3_training.txt", "#" + sentences.strip() + "\n").split("\n")
            #print("res:", cazy, raw_text, res)

            for r in res:
                results = r.split("|")
                #print (results)
                content = ""
                if len(results) == 2 and results[0] != "COEXISTS" and results[0] != "RELATES":
                    #print ("hello")
                    #print (f"{doi}|{is_primary}|{results[0]}|{results[1]}|{results[2]}")
                    #print ("results[1]", results[1])
                    disam = entity_linkage.name_disambiguation(results[1])
                    if disam != "":
                        results[1] = disam

                    content = f"{cazy}\t{sentences.strip()}\t{cazy}|" + "|".join(results) + "\n"
                else:
                    content = f"{cazy}\t{sentences.strip()}\t{cazy}|" + "|".join(results) + "\n"
                
                nonsensical_relates = False
                if results[0] == "RELATES":
                    if results[1] == cazy:
                        nonsensical_relates = True
                    if results[1] not in sentences:
                        nonsensical_relates = True

                if nonsensical_relates == False:
                    activity_output = open("./raw_data/cazy_gpt3_extract.tsv", 'a')
                    activity_output.write(content)
                    activity_output.close()


In [52]:
is_header = False
for line in open("./raw_data/cazy_gpt3_extract.tsv", 'r'):
    if is_header == False:
        is_header = True
    else:
        relations = line.strip().split("\t")[2]

        fields = relations.split("|")
        #print (fields)
        subject = fields[0]
        verb = fields[1]
        object = fields[2]

        activity_output = open("./raw_data/cazy_subfamily_relation.tsv", 'a')
        activity_output.write(f'{subject}\t{object}\t{verb}\n')
        activity_output.close()

In [55]:
import sys
import pandas as pd
import os

#input_file = sys.argv[1]

input_file = "./raw_data/cazy_subfamily_relation.tsv"

df = pd.read_csv(input_file, sep='\t')

df["doi"] = "CAZy"

df["primary"] = 1

subfiles = df.action.unique()


    #print (df[df.action == s])
    
    #newfile = os.path.join(output_folder, s + ".tsv")

df.loc[df.action == "BINDS", "to"] = df[df.action == "BINDS"]["to"].str.lower()

df.to_csv("./raw_data/cazy_subfamily_relation_lower.tsv", sep="\t", index=False) 

In [10]:
import sys
import pandas as pd
import os

#input_file = sys.argv[1]

input_file = "./data_for_neo4j_bacteroidetes/BINDS.tsv"

df = pd.read_csv(input_file, sep='\t')

sugar = df.to.unique()
sugar = set(list(sugar))
sugar

{'alpha-1,3-glucan',
 'alpha-glucans',
 'amylopectin',
 'amylose',
 'arabinofuranose',
 'arabinogalactan',
 'beta-1,3-1,4-glucan',
 'beta-1,3-1,4-mixed linked glucans',
 'beta-1,3-glucan',
 'beta-1,4-galactan',
 'beta-1,4-glucan',
 'beta-1,4-glucans',
 'beta-1,6-glucan',
 'beta-galactan',
 'beta-glucans',
 'cellooligosaccharides',
 'cellulose',
 'chitin',
 'cyclodextrins',
 'fructans',
 'fucose',
 'galactomannan',
 'galactosaminogalactan',
 'galactose',
 'glucomannan',
 'glycogen',
 'hyaluronic acid',
 'inulin',
 'l-rhamnose',
 'lactose',
 'lipopolysaccharides',
 'lipoteichoic acid',
 'maltoheptaose',
 'maltotetraose',
 'maltotriose',
 'mannans',
 'mannose',
 'n-acetyl-lactosamine',
 'n-acetylgalactosamine',
 'non-crystalline cellulose',
 'pectins',
 'peptidoglycan',
 'polygalacturonic acid',
 'pullulan',
 'starch',
 'ulvan',
 'xanthan',
 'xylan',
 'xylans',
 'xyloglucan',
 'xylooligosaccharides'}

In [12]:
import sys
import pandas as pd
import os

#input_file = sys.argv[1]

input_file = "./data_for_neo4j_bacteroidetes/cazy_literature_extract.tsv"

df = pd.read_csv(input_file, sep='\t')

sugar_1 = df.substrate.unique()
sugar_1 = set(list(sugar_1))
sugar_1


{'beta-glucans',
 'cellulose',
 'fucoidan ',
 'glucomannan',
 'starch',
 'xyloglucan'}

In [16]:
content = "substrate\n" + "\n".join(list(sugar.union(sugar_1)))

sugar_output = open("./data_for_neo4j_bacteroidetes/sugar.tsv", 'a')
sugar_output.write(content)
sugar_output.close()